# Construire une application à base de GML

## Simple appel à un GML

In [ ]:
! ollama pull mixtral:8x7b

In [1]:
%pip install -qU langchain-ollama==0.2.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="mixtral:8x7b",
    temperature=0,
    # other params...
)

In [4]:
from langchain_core.messages import AIMessage

messages = [
    (
        "system",
        "Translate the following from English into French.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content=' Here is the translation of "I love programming" in French:\n\nJ\'aime programmer.\n\nPlease note that the verb "to love" should be translated as "aimer" in French, and "programming" can be translated as "programmer". So, when you put them together, you get "J\'aime programmer", which means "I love programming" in French.', additional_kwargs={}, response_metadata={'model': 'mixtral:8x7b', 'created_at': '2024-10-20T14:50:10.893381Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3471792709, 'load_duration': 21765584, 'prompt_eval_count': 21, 'prompt_eval_duration': 1116722000, 'eval_count': 86, 'eval_duration': 2330773000}, id='run-7266478f-ac08-4075-b815-00460f6dbece-0', usage_metadata={'input_tokens': 21, 'output_tokens': 86, 'total_tokens': 107})

In [5]:
print(ai_msg.content)

 Here is the translation of "I love programming" in French:

J'aime programmer.

Please note that the verb "to love" should be translated as "aimer" in French, and "programming" can be translated as "programmer". So, when you put them together, you get "J'aime programmer", which means "I love programming" in French.


## Analyse d'image

In [6]:
import base64

import httpx

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama

model = ChatOllama(model="bakllava", temperature=0)

def prompt_func(data):
    text = data["text"]
    image = data["image"]

    image_part = {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{image}",
    }

    content_parts = []

    text_part = {"type": "text", "text": text}

    content_parts.append(image_part)
    content_parts.append(text_part)

    return [HumanMessage(content=content_parts)]


from langchain_core.output_parsers import StrOutputParser

chain = prompt_func | model | StrOutputParser()

query_chain = chain.invoke(
    {"text": "Describe the image provided", "image": image_data}
)

print(query_chain)


 The image features a wooden walkway or bridge that is surrounded by lush green grass on both sides. The path appears to lead towards a beautiful wildflower field, creating a serene and picturesque scene. In the background, there are trees that further enhance the natural beauty of the area. Overall, it's an inviting and peaceful location for a stroll or a moment of relaxation amidst nature.


## Développer un RAG

In [15]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [12]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="llama3",
)

In [13]:
# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_documents = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_documents[0].page_content

'LangChain is the framework for building context-aware reasoning applications'

In [16]:

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

/Users/magnin/miniconda3/envs/mgl7320/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


" Task Decomposition is a process of breaking down complex tasks into smaller, manageable sub-tasks. In the context provided, it refers to the AI assistant's capability to parse user input into multiple tasks, each with a specific function and dependency on other tasks. This allows the assistant to efficiently manage and execute these tasks, ensuring a more accurate and reliable response."